In [1]:
from __future__ import print_function

## GSL Pythonization Tutorial

_(Hat tip to Neil Dhir for the idea.)_

This tutorial introduces pythonizations and how they can be used to solve low-level problems.

The setup: imagine you want to use numpy, but are given a C or C++ library that is based on the GNU Scientific Library (GSL). How do make the two of them play nice together?

GSL is written in C, but is very structured: it has consistent naming conventions, provides allocators/deallocators for its main structs, and has clear ownership rules. Because of this structure, it is possible to write pythonizations based on reflection information that end up being very simple and therefore easy to maintain.

In [2]:
import cppyy

For simplicity, we will use `gsl_blas_dgemm` as a stand-in for the "C/C++ library based on GSL." To make our life easier, we will wrap up the bindings to GSL and GSLBLAS into a single reflection dictionary. This is overkill for simple projects, but if we want access to all of GSL (as opposed to GSL and GSLBLAS separately, say), this approach is fine.

In [3]:
# first, pull in all headers from the GSL installation directory (/usr/include on my system).
import glob, os
GSL_HOME = '/usr/include'
gsl_headers = [os.path.relpath(x, GSL_HOME) for x in glob.glob(GSL_HOME+'/gsl/*.h')]

Next, we write a selection file that picks up all interesting bits from GSL. This file can be kept simple because of the proper naming conventions (it could have been simpler still if GSL were a C++ library, living in a single namespace). We then run the genreflex command to generate the dictionary file and compile it, linking in GSL and GSLBLAS. Finally, we're ready to load the dictionary reflection file into cppyy.

In [4]:
%%file gsl_selection.xml
<lcgdict>
   <struct pattern="gsl_*" />
   <function pattern="gsl_*" />
   <enum pattern="GSL*" />
   <enum pattern="CBLAS*" />
</lcgdict>


Overwriting gsl_selection.xml


In [5]:
# conventional name for generated output
rfldct = 'GSLDict'

if not os.path.exists('%s_rflx.cpp' % rfldct):
    import subprocess, sys

    # generate the reflection dictionary
    try:
        subprocess.check_output(
            ['genreflex',                  # utility installed by pip when installing cppyy
             '-s', 'gsl_selection.xml',    # selection file (see above)
             '-o', '%s_rflx.cpp'%rfldct,   # intermediate output file
             '-I'+GSL_HOME]+               # include search path for GSL headers
             gsl_headers)                  # headers themselves
    except subprocess.CalledProcessError as e:
        print("genreflex failed (%d):" % e.returncode, e.output)
    else:
        print("genreflex done")

genreflex done


In [6]:
if not os.path.exists('%s.so' % rfldct):
    # get command line arguments for compiler from cling
    try:
        clingflags = subprocess.check_output(
            ['cling-config',               # utility installed by pip when installing cppyy
             '--cppflags'])
    except subprocess.CalledProcessError as e:
        print('cling-config failed (%d):' % e.returncode, e.output)
        raise
    else:
        print('cling-config done')

    # compile generated file
    try:
        subprocess.check_output(
            ['g++',                        # C++ compiler
             '-fPIC',                      # require position independent code
             '-shared',                    # generate shared library
             '-o', '%s.so'%rfldct,         # output file
             '-I'+GSL_HOME,                # include search path for GSL headers
             '%s_rflx.cpp'%rfldct]+        # intermediate file to compile
             clingflags.split()+           # extra flags provided by cling
             ['-lgsl', '-lgslcblas'])      # link in GSL and GSLBLAS
    except subprocess.CalledProcessError as e:
        print('compilation failed (%d):' % e.returncode, e.output)
    else:
        print('compilation done')

cling-config done
compilation done


In [7]:
# load the generated dictionary
cppyy.load_reflection_info(rfldct)